In [1]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.backends.cudnn
import polars as pl
import pandas as pd
import numpy as np
import seaborn as sns
import warnings

from datetime import datetime, timedelta
from torch_geometric.nn import GATv2Conv, GATConv
from torch_geometric.utils import dense_to_sparse
from torch.distributions import Normal, Laplace, RelaxedOneHotCategorical
from torchdiffeq import odeint  # For continuous-time normalizing flows

from feature.scalers import ranged_scaler
from feature.engineering import *
from CARAT.model_utils import *
from CARAT.model import CausalGraphVAE
from CARAT.components import *
from utils.utils import set_seed, logger

# Torch settings"""
"""torch.use_deterministic_algorithms(False)
torch.backends.cudnn.benchmark = False
torch.autograd.profiler.profile(enabled=False)
torch.autograd.profiler.emit_nvtx(enabled=False)
torch.autograd.set_detect_anomaly(mode=False)

# Environment variables
os.environ['CUBLAS_WORKSPACE_CONFIG'] = '167772160'
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
os.environ['TORCH_USE_CUDA_DSA'] = "1"
"""
# Set device
device = torch.device('cpu')#torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Suppress warnings
warnings.filterwarnings('ignore')

In [2]:
cats_df = pl.read_csv("data/data.csv", separator=",")  
print(cats_df.shape)
metadata = pl.read_csv('data/metadata.csv',separator=',')
potential_causes = metadata['root_cause'].unique().to_list()

for col in cats_df.columns:
    unique_vals = cats_df[col].n_unique()
    data_type = cats_df[col].dtype
    bad_dtypes = [pl.Date,pl.Datetime,pl.Utf8]
    if ((unique_vals >= 50) & (data_type not in bad_dtypes) ):
        cats_df = cats_df.with_columns(ranged_scaler(cats_df[col]))
    else:
        continue

(5000000, 20)


In [3]:
cats_df = cats_df.with_columns(
    pl.col('timestamp').str.to_datetime("%Y-%m-%d %H:%M:%S"),
    pl.Series("entity_id",range(len(cats_df)))
)

In [4]:
cats_rows_list = metadata.rows(named=True)
cats_df.shape

(5000000, 21)

In [5]:
cats_df = make_stationary(cats_df)

⚠️ Skipping column timestamp (not a float column)
Column: aimp | Final ADF: -2211.6294 | p-value: 0.0000 | Diffs: 0
Column: amud | Final ADF: -8.0091 | p-value: 0.0000 | Diffs: 0
Column: arnd | Final ADF: -18.1248 | p-value: 0.0000 | Diffs: 0
Column: asin1 | Final ADF: -5.7565 | p-value: 0.0000 | Diffs: 1
Column: asin2 | Final ADF: -39.8976 | p-value: 0.0000 | Diffs: 3
Column: adbr | Final ADF: -111.5305 | p-value: 0.0000 | Diffs: 0
Column: adfl | Final ADF: -217.2031 | p-value: 0.0000 | Diffs: 0
Column: bed1 | Final ADF: -35.0214 | p-value: 0.0000 | Diffs: 0
Column: bed2 | Final ADF: -44.5328 | p-value: 0.0000 | Diffs: 0
Column: bfo1 | Final ADF: -8.5614 | p-value: 0.0000 | Diffs: 0
Column: bfo2 | Final ADF: -13.3015 | p-value: 0.0000 | Diffs: 0
Column: bso1 | Final ADF: -2164.1026 | p-value: 0.0000 | Diffs: 1
Column: bso2 | Final ADF: -13.5707 | p-value: 0.0000 | Diffs: 0
Column: bso3 | Final ADF: -14.7270 | p-value: 0.0000 | Diffs: 0
Column: ced1 | Final ADF: -727.3289 | p-value: 0.

In [6]:
cats_df = cats_df.to_pandas()

In [7]:
metadata['affected'].unique().to_list()

["['cso1']", "['cfo1']", "['ced1']"]

In [8]:
potential_causes = metadata['root_cause'].unique().to_list()

In [9]:
cats_df=cats_df.set_index('timestamp')
cats_df = cats_df.drop(['y','category','entity_id'],axis=1)
cats_df.head()


,aimp,amud,arnd,asin1,asin2,adbr,adfl,bed1,bed2,bfo1,bfo2,bso1,bso2,bso3,ced1,cfo1,cso1
timestamp,,,,,,,,,,,,,,,,,
2023-01-01 00:00:06,0.0,0.142857,-0.444480,0.00002,-7.999823e-12,0.0,0.0,-0.32802,-0.369237,-0.738163,-0.767181,0.000072,-0.507953,-0.716059,0.0,0.100401,-0.186461
2023-01-01 00:00:07,0.0,0.142857,-0.446078,0.00002,-7.999823e-12,0.0,0.0,-0.32802,-0.369237,-0.738163,-0.767181,0.000083,-0.507953,-0.716059,0.0,0.100408,-0.186406
2023-01-01 00:00:08,0.0,0.142857,-0.447166,0.00002,-8.000267e-12,0.0,0.0,-0.32802,-0.369237,-0.738163,-0.767181,0.000094,-0.507953,-0.716059,0.0,0.100416,-0.186345
2023-01-01 00:00:09,0.0,0.142857,-0.442843,0.00002,-7.999823e-12,0.0,0.0,-0.32802,-0.369237,-0.738163,-0.767181,0.000104,-0.507953,-0.716059,0.0,0.100426,-0.186278
2023-01-01 00:00:10,0.0,0.142857,-0.441320,0.00002,-8.000045e-12,0.0,0.0,-0.32802,-0.369237,-0.738163,-0.767181,0.000113,-0.507953,-0.716059,0.0,0.100438,-0.186205


In [10]:
cats_df.shape

(4999994, 17)

In [11]:
train_df = cats_df[0:1000000]
test_df = cats_df[1000000:]

In [12]:
test_df = test_df.astype('float32')

In [13]:
try:
    train_df = train_df.drop('time',axis=1)
except:
    None
try:
    test_df = test_df.drop('time',axis=1)
except:
    None

In [14]:
cols = list(test_df.columns)
non_causal_columns = list(set(cols).difference(set(potential_causes)))
causal_indices = [train_df.columns.get_loc(col) for col in potential_causes]
non_causal_indices = [train_df.columns.get_loc(col) for col in non_causal_columns]

In [15]:
import torch
import random
import numpy as np

# Define the seed value
seed = 42

# Set seed for PyTorch
torch.manual_seed(seed)

# Set seed for CUDA (if using GPUs)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)  # For multi-GPU setups

# Set seed for Python's random module
random.seed(seed)

# Set seed for NumPy
np.random.seed(seed)

# Ensure deterministic behavior for PyTorch operations
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [ ]:
TIME_STEPS = 3
BATCH_SIZE = 10000
hidden_dim = 128
latent_dim = 8
num_nodes = 17

dataset_nominal = TimeSeriesDataset(train_df, device=device, time_steps=TIME_STEPS)
dataloader_nominal = DataLoader(dataset_nominal, batch_size=BATCH_SIZE, shuffle=False, pin_memory=False)

# Initialize model and optimizer
model = CausalGraphVAE(input_dim=train_df.shape[1], hidden_dim=hidden_dim,
                        latent_dim=latent_dim, num_nodes=train_df.shape[1],device=device,
                        time_steps=TIME_STEPS, prior_adj=None)
optimizer = torch.optim.AdamW(model.parameters(), lr=0.01, weight_decay=1e-10)

# Train on nominal data
print("Pretraining on nominal data...")
model.train_model(dataloader_nominal, optimizer, num_epochs=250, patience=30,rho_max=0.2,alpha_max=0.2)


Pretraining on nominal data...
Epoch 1: Loss = 21468.7776
Recon Loss =4894.2446, KL Loss = 2.4268, Sparsity Loss = 116.1447, Lagrangian Loss = 320.6222


In [ ]:
prior_adj = model.causal_graph.adj_mat.clone().detach()
#scaled_prior = scale_tensor(prior_adj)
for i, row in enumerate(prior_adj):
    for j, column in enumerate(row):
        if (j in non_causal_indices) and (i in causal_indices) & (i!=j):
            continue
        else:
            prior_adj[i,j] = 0

In [ ]:
pd.DataFrame(prior_adj.cpu().detach().numpy(),index=cols,columns=cols)

In [23]:
cols = list(test_df.columns)
non_causal_columns = list(set(cols).difference(set(potential_causes)))
causal_indices = [train_df.columns.get_loc(col) for col in potential_causes]
non_causal_indices = [train_df.columns.get_loc(col) for col in non_causal_columns]
num_nodes = len(train_df.columns)

new_metadata = []
hidden_dim = 64

edge_correct = 0
instantaneous_correct = 0
lagged_correct = 0
counterfactual_correct = 0 
rr_correct = 0
total_correct = 0
total_checked = 0
incorrect = []

for i, row in enumerate(cats_rows_list):
    total_checked +=1 
    logger.info('Model: '+ str(i))
    anomaly = eval(row['affected'])[0]
    print('Anomaly: ' + anomaly)
    anomaly_time = datetime.strptime(row['start_time'],"%Y-%m-%d %H:%M:%S")
    #start_time = datetime.strptime(row['start_time'],"%Y-%m-%d %H:%M:%S")
    end_time = datetime.strptime(row['end_time'],"%Y-%m-%d %H:%M:%S")
    root_cause = row['root_cause']
    #start_len = mod_df.shape[0]
    #if start_len >1000:
        #start_len = 1000
    start_len = 50
    start_time = anomaly_time- timedelta(seconds=start_len)
    finish_time = end_time + timedelta(seconds=start_len)
    mod_df = test_df[(test_df.index>= start_time) & (test_df.index<= finish_time)]
    mod_df = mod_df[['aimp', 'amud', 'arnd', 'asin1', 'asin2', 'adbr', 'adfl', 'bed1',
       'bed2', 'bfo1', 'bfo2', 'bso1', 'bso2', 'bso3', 'ced1', 'cfo1', 'cso1']]

    """
    FIND THE OPTIMAL NUMBER OF LAGS
    """
    TIME_STEPS = max(most_frequent(find_optimal_lags_for_dataframe(mod_df))+1,3)

    dataset = TimeSeriesDataset(mod_df,device=device, time_steps=TIME_STEPS)
    dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=False)
    X_data = torch.empty(0,device=device)
    T_data = torch.empty(0,device=device)
    for batch_idx, (X_batch, time_batch) in enumerate(dataloader):
        X_data = torch.cat((X_data[:batch_idx], X_batch, X_data[batch_idx:]))
        T_data = torch.cat((T_data[:batch_idx], time_batch, T_data[batch_idx:]))
    
    fine_tuned = CausalGraphVAE(input_dim=num_nodes, hidden_dim=hidden_dim,
                           latent_dim=latent_dim, num_nodes=num_nodes,device=device,
                           time_steps=TIME_STEPS,prior_adj=prior_adj)
    optimizer = torch.optim.AdamW(fine_tuned.parameters(), lr=0.01, weight_decay=1e-10)  # L2 Regularization

    
    fine_tuned.train_model(dataloader, optimizer, num_epochs=2500, patience=50,BATCH_SIZE=BATCH_SIZE,rho_max=0.2,alpha_max=0.2)

    causes = fine_tuned.infer_causal_effect(X_data.to(torch.float32).to(device),T_data.to(torch.float32).to(device),anomaly,cols,non_causal_indices=non_causal_indices)
    
    causes = causes.filter(items=potential_causes, axis=0)
    edge_cause_1 = causes.sort_values(by='causes',ascending=False)[0:3].index[0]
    edge_cause_2 = causes.sort_values(by='causes',ascending=False)[0:3].index[1]
    edge_cause_3 = causes.sort_values(by='causes',ascending=False)[0:3].index[2]
    
    instant_cause_1 = causes.sort_values(by='instantaneous',ascending=False)[0:3].index[0]
    instant_cause_2 = causes.sort_values(by='instantaneous',ascending=False)[0:3].index[1]
    instant_cause_3 = causes.sort_values(by='instantaneous',ascending=False)[0:3].index[2]
    
    lag_cause_1 = causes.sort_values(by='lagged',ascending=False)[0:3].index[0]
    lag_cause_2 = causes.sort_values(by='lagged',ascending=False)[0:3].index[1]
    lag_cause_3 = causes.sort_values(by='lagged',ascending=False)[0:3].index[2]
    
    counterfactual_cause_1 = causes.sort_values(by='counterfactuals',ascending=False)[0:3].index[0]
    counterfactual_cause_2 = causes.sort_values(by='counterfactuals',ascending=False)[0:3].index[1]
    counterfactual_cause_3 = causes.sort_values(by='counterfactuals',ascending=False)[0:3].index[2]

    rr_cause_1 = causes.sort_values(by='RootRank',ascending=False)[0:3].index[0]
    rr_cause_2 = causes.sort_values(by='RootRank',ascending=False)[0:3].index[1]
    rr_cause_3 = causes.sort_values(by='RootRank',ascending=False)[0:3].index[2]
    
    total_score_cause_1=causes.sort_values(by='causal_strength',ascending=False)[0:3].index[0]
    total_score_cause_2=causes.sort_values(by='causal_strength',ascending=False)[0:3].index[1]
    total_score_cause_3=causes.sort_values(by='causal_strength',ascending=False)[0:3].index[2]
    
    if root_cause == edge_cause_1:
        row['edge_cause_1'] = 1
    if root_cause == edge_cause_1:
        row['edge_cause_2'] = 1
    if root_cause == edge_cause_1:
        row['edge_cause_3'] = 1
    
    if root_cause == counterfactual_cause_1:
        row['counterfactual_cause_1'] = 1
    if root_cause == counterfactual_cause_2:
        row['counterfactual_cause_2'] = 1
    if root_cause == counterfactual_cause_3:
        row['counterfactual_cause_3'] = 1
    
    if root_cause == total_score_cause_1:
        row['total_score_cause_1'] = 1
    if root_cause == total_score_cause_2:
        row['total_score_cause_2'] = 1
    if root_cause == total_score_cause_3:
        row['total_score_cause_3'] = 1
    
    if root_cause == instant_cause_1:
        row['instant_cause_1'] = 1
    if root_cause == instant_cause_2:
        row['instant_cause_2'] = 1
    if root_cause == instant_cause_3:
        row['instant_cause_3'] = 1
    
    if root_cause == lag_cause_1:
        row['lag_cause_1'] = 1
    if root_cause == lag_cause_2:
        row['lag_cause_2'] = 1
    if root_cause == lag_cause_3:
        row['lag_cause_3'] = 1

    if root_cause == rr_cause_1:
        row['rr_cause_1'] = 1
    if root_cause == rr_cause_2:
        row['rr_cause_2'] = 1
    if root_cause == rr_cause_3:
        row['rr_cause_3'] = 1
    
    new_metadata.append(row)
    
    if root_cause in [total_score_cause_1 , total_score_cause_2 , total_score_cause_3]:
        total_correct+=1
    if root_cause in [edge_cause_1 , edge_cause_2 , edge_cause_3]:
        edge_correct+=1
    if root_cause in [counterfactual_cause_1 , counterfactual_cause_2 , counterfactual_cause_3]:
        counterfactual_correct+=1
    if root_cause in [instant_cause_1 , instant_cause_2 , instant_cause_3]:
        instantaneous_correct+=1
    if root_cause in [lag_cause_1 , lag_cause_2 , lag_cause_3]:
        lagged_correct+=1
    if root_cause in [rr_cause_1 , rr_cause_2 , rr_cause_3]:
        rr_correct+=1
    
    total_accuracy = total_correct/total_checked* 100
    edge_accuracy = edge_correct/total_checked* 100
    cf_accuracy = counterfactual_correct/total_checked* 100
    instant_accuracy = instantaneous_correct/total_checked* 100
    lag_accuracy = lagged_correct/total_checked* 100
    rr_accuracy = rr_correct/total_checked* 100
    
    if root_cause not in [total_score_cause_1 , total_score_cause_2 , total_score_cause_3,edge_cause_1 , edge_cause_2 , edge_cause_3 ]:
        incorrect.append(i) 
    logger.info(f"Edge Accuracy = {edge_accuracy:.2f}, Instantaneous Accuracy = {instant_accuracy:.2f}, Lagged Accuracy = {lag_accuracy:.2f}, Counterfactual Accuracy = {cf_accuracy:.2f},  Blended Accuracy = {total_accuracy:.2f},  RR Accuracy = {rr_accuracy:.2f}  ") 



2025-03-19 15:35:19,446 INFO -- Model: 0


Anomaly: cfo1
Epoch 1: Loss = 7052.2529
Recon Loss =6621.9775, KL Loss = 0.0064, Sparsity Loss = 120.7634, Lagrangian Loss = 309.5058
Epoch 51: Loss = 764.5002
Recon Loss =265.7029, KL Loss = 2.3069, Sparsity Loss = 117.5936, Lagrangian Loss = 378.8969
Epoch 101: Loss = 641.3278
Recon Loss =64.9545, KL Loss = 4.1509, Sparsity Loss = 121.1997, Lagrangian Loss = 451.0227
Epoch 151: Loss = 369.8432
Recon Loss =58.1800, KL Loss = 4.2286, Sparsity Loss = 115.5553, Lagrangian Loss = 191.8793
Epoch 201: Loss = 308.3878
Recon Loss =62.1439, KL Loss = 4.1178, Sparsity Loss = 110.1937, Lagrangian Loss = 131.9324
Epoch 251: Loss = 287.8751
Recon Loss =69.8216, KL Loss = 4.0854, Sparsity Loss = 105.9454, Lagrangian Loss = 108.0227
Epoch 301: Loss = 269.3845
Recon Loss =51.0238, KL Loss = 3.9421, Sparsity Loss = 106.5770, Lagrangian Loss = 107.8416
Early stopping triggered. Last Epoch: 342
Recon Loss =51.4998, KL Loss = 3.8687, Sparsity Loss = 108.0268, Lagrangian Loss = 109.9689


2025-03-19 15:35:54,572 INFO -- Edge Accuracy = 100.00, Instantaneous Accuracy = 100.00, Lagged Accuracy = 0.00, Counterfactual Accuracy = 0.00,  Blended Accuracy = 100.00,  RR Accuracy = 0.00  
2025-03-19 15:35:54,573 INFO -- Model: 1


Anomaly: cfo1
Epoch 1: Loss = 57012.4414
Recon Loss =56367.6250, KL Loss = 0.0086, Sparsity Loss = 126.0190, Lagrangian Loss = 518.7902
Epoch 51: Loss = 5966.3086
Recon Loss =5382.8242, KL Loss = 2.5729, Sparsity Loss = 122.0903, Lagrangian Loss = 458.8214
Epoch 101: Loss = 1703.6772
Recon Loss =1392.7463, KL Loss = 4.4104, Sparsity Loss = 113.7722, Lagrangian Loss = 192.7483
Epoch 151: Loss = 1769.7041
Recon Loss =1307.6007, KL Loss = 4.3660, Sparsity Loss = 118.3791, Lagrangian Loss = 339.3585
Early stopping triggered. Last Epoch: 159
Recon Loss =1347.8794, KL Loss = 4.4392, Sparsity Loss = 119.9643, Lagrangian Loss = 358.2567


2025-03-19 15:37:06,845 INFO -- Edge Accuracy = 50.00, Instantaneous Accuracy = 50.00, Lagged Accuracy = 0.00, Counterfactual Accuracy = 0.00,  Blended Accuracy = 50.00,  RR Accuracy = 50.00  
2025-03-19 15:37:06,846 INFO -- Model: 2


...

2025-03-19 17:04:38,233 INFO -- Edge Accuracy = 37.25, Instantaneous Accuracy = 35.29, Lagged Accuracy = 42.48, Counterfactual Accuracy = 58.17,  Blended Accuracy = 43.14,  RR Accuracy = 50.33  
2025-03-19 17:04:38,234 INFO -- Model: 153


Anomaly: cso1
Epoch 1: Loss = 10170.3232
Recon Loss =9683.3564, KL Loss = 0.0071, Sparsity Loss = 123.8399, Lagrangian Loss = 363.1197
Epoch 51: Loss = 985.4464
Recon Loss =479.6533, KL Loss = 2.1088, Sparsity Loss = 119.6129, Lagrangian Loss = 384.0714
Epoch 101: Loss = 745.7746
Recon Loss =137.1000, KL Loss = 3.8521, Sparsity Loss = 122.0956, Lagrangian Loss = 482.7269
Epoch 151: Loss = 455.7894
Recon Loss =97.6257, KL Loss = 4.0351, Sparsity Loss = 115.4751, Lagrangian Loss = 238.6535
Epoch 201: Loss = 367.5910
Recon Loss =90.0719, KL Loss = 4.0005, Sparsity Loss = 116.4643, Lagrangian Loss = 157.0544
Epoch 251: Loss = 324.6081
Recon Loss =91.4886, KL Loss = 3.9154, Sparsity Loss = 109.8417, Lagrangian Loss = 119.3624
Epoch 301: Loss = 328.5468
Recon Loss =96.7554, KL Loss = 3.9920, Sparsity Loss = 109.3706, Lagrangian Loss = 118.4288
Early stopping triggered. Last Epoch: 323
Recon Loss =96.4067, KL Loss = 3.9195, Sparsity Loss = 104.5991, Lagrangian Loss = 103.1000


2025-03-19 17:05:15,785 INFO -- Edge Accuracy = 37.66, Instantaneous Accuracy = 35.71, Lagged Accuracy = 42.86, Counterfactual Accuracy = 58.44,  Blended Accuracy = 43.51,  RR Accuracy = 50.65  
2025-03-19 17:05:15,786 INFO -- Model: 154


Anomaly: cso1
Epoch 1: Loss = 11935.3447
Recon Loss =11539.5518, KL Loss = 0.0126, Sparsity Loss = 121.2086, Lagrangian Loss = 274.5711
Epoch 51: Loss = 1039.0872
Recon Loss =560.4792, KL Loss = 2.7717, Sparsity Loss = 119.3773, Lagrangian Loss = 356.4590
Epoch 101: Loss = 568.6068
Recon Loss =93.4997, KL Loss = 3.8985, Sparsity Loss = 117.7506, Lagrangian Loss = 353.4580
Epoch 151: Loss = 376.0942
Recon Loss =92.9102, KL Loss = 4.3304, Sparsity Loss = 117.7408, Lagrangian Loss = 161.1128


KeyboardInterrupt: 